In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
!pip install geojson

In [ ]:
import ee
import json
import geojson
from geojson import Feature, Point, FeatureCollection

# Initialize the Earth Engine API
ee.Initialize()
longitude_min =  100.55194295234655
latitude_min = 13.733571685415455
longitude_max = 100.5580444434391
latitude_max =13.72573111966259

# Define the area of interest (you can change this to any coordinates of a specific region)
area_of_interest = ee.Geometry.Rectangle([longitude_min, latitude_min, longitude_max, latitude_max])

# Use the Sentinel-2 satellite imagery
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR")

# Filter the image collection by date and area of interest
filtered_collection = sentinel2.filterDate('2020-01-01', '2020-12-31').filterBounds(area_of_interest)

# Check if the filtered collection is empty
if filtered_collection.size().getInfo() == 0:
    print("No images found in the specified date range and area of interest.")
else:
    # Select the first image from the filtered collection
    image = filtered_collection.first()

    # Normalized Difference Vegetation Index (NDVI) threshold (you can adjust this value based on your needs)
    ndvi_threshold = 0.3

    # Calculate NDVI using NIR (B8) and Red (B4) bands
    ndvi = image.normalizedDifference(['B8', 'B4'])

    # Create a binary image by setting pixels with NDVI greater than the threshold to 1, and others to 0
    binary_image = ndvi.gt(ndvi_threshold)

    # Apply connected pixel algorithm to filter out small patches of trees
    connected_pixel_image = binary_image.connectedPixelCount(100, True).eq(100)

    # Vectorize the connected pixel image
    vectorized_image = connected_pixel_image.reduceToVectors(
        geometry=area_of_interest,
        scale=10,
        maxPixels=1e9,
        geometryType='centroid',
        eightConnected=False,
        labelProperty='tree'
    )

    # Convert the vectorized image to GeoJSON
    features = vectorized_image.getInfo()["features"]
    geojson_data = geojson.FeatureCollection(features)

    # Save the GeoJSON data to a file
    with open("tree_locations.geojson", "w") as f:
        json.dump(geojson_data, f)
